In [2]:
!qiime --version

q2cli version 2021.8.0
Run `qiime info` for more version details.


In [1]:
import pandas as pd
import qiime2
from qiime2 import Artifact
import os
from qiime2.plugins import diversity, feature_table, fragment_insertion, feature_classifier, metadata, emperor, deicode, gneiss, taxa

In [2]:
tbl_filt = Artifact.load('./data/preprocessed-feature-table-4k-only-apt.qza')
tbl_unrare_filt = Artifact.load('./data/preprocessed-feature-table-unrare-only-apt.qza')
meta = qiime2.Metadata.load('./data/13957_metadata.txt')
tree = Artifact.load('./data/filt_tree.qza')
tax = Artifact.load('./data/137124-reference-hit.taxonomy_gg.qza')

Before I make the PCoAs I need to filter out the samples not associated with a living space. I can filter out from 'indoor_space_classsifier' with dont_use values

In [40]:
tbl_filt.view(pd.DataFrame).sum(axis=1)

13957.Apt.A.kitchen.cabinet.face.1               4000.0
13957.Apt.A.bathroom.sink.handle.left            4000.0
13957.Apt.A.bedroom.door.face.inside             4000.0
13957.Apt.A.kitchen.cabinet.handle.7             4000.0
13957.Apt.A.kitchen.fridge.floor                 4000.0
                                                  ...  
13957.Apt.C.toilet.bathroom.toilet.rim           4000.0
13957.Apt.C.toilet.bathroom.toilet.seat          4000.0
13957.Apt.C.toilet.bathroom.toilet.tank.cover    4000.0
13957.Apt.C.toilet.bathroom.wall.far             4000.0
13957.Apt.C.toilet.bathroom.wall.near            4000.0
Length: 224, dtype: float64

In [3]:
# Note I later went back and did this in the diversity_analysis_preprocessing notebook
# they are saved as preprocessed-feature-table-4k-only-apt.qza and 
# preprocessed-feature-table-unrare-only-apt.qza. the tbl_filt and tbl_unrare_filt should be
# exactly the same as just loading those
#tbl_filt = feature_table.actions.filter_samples(table = tbl, 
#                                     metadata = meta,
#                                     where = "[indoor_space_classifier]!='dont_use'")

#tbl_unrare_filt = feature_table.actions.filter_samples(table = tbl_unrare, 
#                                     metadata = meta,
#                                     where = "[indoor_space_classifier]!='dont_use'")

## Beta diversity

In [5]:
# making the biodiversity distance matrices
!mkdir -p ./analysis/bdiv
!mkdir -p ./analysis/bdiv/distances
!mkdir -p ./analysis/bdiv/pcoas
!mkdir -p ./analysis/bdiv/emperor
!mkdir -p ./analysis/bdiv/viz

In [15]:
# initializing the list
bdiv_dms = {}

In [16]:
for metric in ('jaccard', 'braycurtis'):
    bdiv_dms[metric] = diversity.pipelines.beta(
        table= tbl_filt, metric=metric).distance_matrix
    
for metric in ('unweighted_unifrac', 'weighted_unifrac'):
    bdiv_dms[metric] = diversity.pipelines.beta_phylogenetic(
        table=tbl_filt, phylogeny=tree, metric=metric).distance_matrix

/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [17]:
for metric in bdiv_dms:
    bdiv_dms[metric].save('./analysis/bdiv/distances/%s.qza' % metric)

pcoas = {k: diversity.methods.pcoa(v).pcoa for k, v in bdiv_dms.items()}

/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.22947217175179552 and the largest is 16.013140607455476.
  warn(
/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section f

In [18]:
for metric in pcoas:
    pcoas[metric].save('./analysis/bdiv/pcoas/%s.pcoa.qza' % metric)

In [19]:
{k: diversity.methods.pcoa(v).pcoa for k, v in bdiv_dms.items()}

{'jaccard': <artifact: PCoAResults uuid: 7f0fa383-52b6-49fc-baf1-7547e9a3c06c>,
 'braycurtis': <artifact: PCoAResults uuid: 0444cc15-d8f3-4f37-b73f-fc554e53639d>,
 'unweighted_unifrac': <artifact: PCoAResults uuid: 7186acd7-a467-4a3c-a83b-a4f7dccb5367>,
 'weighted_unifrac': <artifact: PCoAResults uuid: 5c3a21bb-8dce-474d-a225-4f0e889609bc>}

In [20]:
emplots = {k: emperor.visualizers.plot(v, meta, ignore_missing_samples = True).visualization for k, v in pcoas.items()}

In [21]:
emplots

{'jaccard': <visualization: Visualization uuid: 4a187919-4e83-47ff-a49c-70e1b594b03e>,
 'braycurtis': <visualization: Visualization uuid: cc627c76-06f9-43b2-bd11-e5b88ee6196e>,
 'unweighted_unifrac': <visualization: Visualization uuid: d6b42362-8187-4818-80b2-f4558d16d757>,
 'weighted_unifrac': <visualization: Visualization uuid: 6db15c9e-2818-448c-9c91-093191bef2ba>}

In [22]:
for metric in emplots:
    emplots[metric].save('./analysis/bdiv/emperor/%s.pcoa.qza' % metric)

Beta group significance visualizations are below

## Alpha diversity

In [11]:
core_metr_phyl = diversity.pipelines.core_metrics_phylogenetic(table = tbl_filt,
                                                               sampling_depth = 4000,
                                                               phylogeny = tree,
                                                               metadata = meta,
                                                               n_jobs_or_threads = 2)

/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.22947217175179552 and the largest is 16.013140607455476.
  warn(
/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude

In [15]:
!mkdir -p ./analysis/adiv
!mkdir -p ./analysis/adiv/viz

In [12]:
core_metr_phyl.shannon_vector.save('./analysis/adiv/shannon.qza')
core_metr_phyl.observed_features_vector.save('./analysis/adiv/observed_feat.qza')
core_metr_phyl.evenness_vector.save('./analysis/adiv/evenness.qza')
core_metr_phyl.faith_pd_vector.save('./analysis/adiv/faith_pd.qza')

'./analysis/adiv/faith_pd.qza'

In [13]:
adiv_viz = {}
adiv_viz['shannon'] = diversity.visualizers.alpha_group_significance(
        alpha_diversity = core_metr_phyl.shannon_vector, metadata = meta)

adiv_viz['observed_features'] = diversity.visualizers.alpha_group_significance(
        alpha_diversity = core_metr_phyl.observed_features_vector, metadata = meta)

adiv_viz['evenness'] = diversity.visualizers.alpha_group_significance(
        alpha_diversity = core_metr_phyl.evenness_vector, metadata = meta)

adiv_viz['faith_pd'] = diversity.visualizers.alpha_group_significance(
        alpha_diversity = core_metr_phyl.faith_pd_vector, metadata = meta)

In [14]:
for metric in ('shannon', 'observed_features', 'evenness', 'faith_pd'):
    adiv_viz[metric].visualization.save('./analysis/adiv/viz/%s.qzv' % metric)

### Bdiv group significance

In [3]:
#calculating group significance for detected + inconclusive vs not detected
bdiv_dm = {}
bdiv_viz_anova = {}
bdiv_viz_disp = {}
for metric in ('unweighted_unifrac', 'weighted_unifrac', 'jaccard', 'braycurtis'):
    bdiv_dm[metric] = Artifact.load('./analysis/bdiv/distances/%s.qza' % metric)
    bdiv_viz_anova[metric] = diversity.visualizers.beta_group_significance(
                        distance_matrix = bdiv_dm[metric], metadata = meta.get_column('indoor_space_detection'), 
                        method = 'permanova')
    bdiv_viz_disp[metric] = diversity.visualizers.beta_group_significance(
                        distance_matrix = bdiv_dm[metric], metadata = meta.get_column('indoor_space_detection'), 
                        method = 'permdisp')

/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.010090904638227195 and the largest is 5.140792755639486.
  warn(
/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section f

In [4]:
for metric in ('unweighted_unifrac', 'weighted_unifrac', 'jaccard', 'braycurtis'):
    bdiv_viz_anova[metric].visualization.save('./analysis/bdiv/viz/%s.anova.room.detect.qzv' % metric)
    bdiv_viz_disp[metric].visualization.save('./analysis/bdiv/viz/%s.disp.room.detect.qzv' % metric)

In [25]:
#calculating group significance for apt + living space
bdiv_viz_anova = {}
bdiv_viz_disp = {}
for metric in ('unweighted_unifrac', 'weighted_unifrac', 'jaccard', 'braycurtis'):
    bdiv_viz_anova[metric] = diversity.visualizers.beta_group_significance(
                        distance_matrix = bdiv_dm[metric], metadata = meta.get_column('apt_space'), 
                        method = 'permanova')
    bdiv_viz_disp[metric] = diversity.visualizers.beta_group_significance(
                        distance_matrix = bdiv_dm[metric], metadata = meta.get_column('apt_space'), 
                        method = 'permdisp')

/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.010090904638227146 and the largest is 5.14079275563949.
  warn(
/Users/vcantu/miniconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section fo

In [26]:
for metric in ('unweighted_unifrac', 'weighted_unifrac', 'jaccard', 'braycurtis'):
    bdiv_viz_anova[metric].visualization.save('./analysis/bdiv/viz/%s.anova.apt_space.qzv' % metric)
    bdiv_viz_disp[metric].visualization.save('./analysis/bdiv/viz/%s.disp.apt_space.qzv' % metric)

## Deicode

In [23]:
!mkdir -p ./analysis/bdiv/deicode

In [41]:
rpca = deicode.methods.rpca(table = tbl_unrare_filt,
                            min_sample_count = 500,
                            min_feature_count = 10)


In [42]:
rpca_biplot = emperor.actions.biplot(biplot = rpca.biplot,
                                    sample_metadata = meta)

In [43]:
rpca.distance_matrix.save('./analysis/bdiv/deicode/deicode_distance.qza')

'./analysis/bdiv/deicode/deicode_distance.qza'

In [44]:
rpca.biplot.save('./analysis/bdiv/deicode/deicode_ordination.qza')

'./analysis/bdiv/deicode/deicode_ordination.qza'

In [45]:
rpca_biplot.visualization.save('./analysis/bdiv/deicode/deicode_biplot.qzv')

'./analysis/bdiv/deicode/deicode_biplot.qzv'

In [46]:
meta.get_column('apt_space')

<CategoricalMetadataColumn name='apt_space' id_count=475>

In [47]:
meta.to_dataframe()['host_subject_id'].value_counts()

Apt.A             150
Apt.C             142
Apt.B             127
Mock.Community     40
Blank              16
Name: host_subject_id, dtype: int64

In [48]:
meta.to_dataframe()['apt_space'].value_counts()

Apt.C.kitchen            52
Apt.A.kitchen            50
Apt.B.kitchen            49
Mock.Community.lab       40
Apt.C.bathroom           36
Apt.A.bathroom           28
Apt.A.bedroom            26
Apt.B.bedroom            25
Apt.A.living_room        25
Apt.B.bathroom           24
Apt.C.bedroom            21
Apt.B.living_room        18
Apt.C.living_room        16
Blank.lab                16
Apt.C.dorm_entrance      12
Apt.A.unused bathroom    12
Apt.B.dorm_entrance       6
Apt.A.dorm_entrance       6
Apt.C.researcher          3
Apt.B.researcher          3
Apt.A.outside             2
Apt.B.outside             2
Apt.C.outside             2
Apt.A.researcher          1
Name: apt_space, dtype: int64

## Subsetted DEICODE

In [30]:
!mkdir -p ./analysis/bdiv/deicode/subject_id
!mkdir -p ./analysis/bdiv/deicode/room

In [49]:
dei_ft = {}
dei = {}
for sub_id in ('a-apt', 'b-apt', 'c-apt'):
    dei_ft[sub_id] = Artifact.load('./data/%s-preprocessed-unrare.qza' % sub_id)
    dei[sub_id] = deicode.methods.rpca(table = dei_ft[sub_id], 
                               min_sample_count = 500,
                               min_feature_count = 10)
    dei[sub_id].distance_matrix.save('./analysis/bdiv/deicode/subject_id/%s-deicode-distance.qza' % sub_id)
    dei[sub_id].biplot.save('./analysis/bdiv/deicode/subject_id/%s-deicode-ordination.qza' % sub_id)

In [50]:
dei_ft = {}
dei = {}
for room in ('kitchen', 'bathroom', 'bedroom', 'living'):
    dei_ft[room] = Artifact.load('./data/%s-preprocessed-unrare.qza' % room)
    dei[room] = deicode.methods.rpca(table = dei_ft[room], 
                               min_sample_count = 500,
                               min_feature_count = 10)
    dei[room].distance_matrix.save('./analysis/bdiv/deicode/room/%s-deicode-distance.qza' % room)
    dei[room].biplot.save('./analysis/bdiv/deicode/room/%s-deicode-ordination.qza' % room)